<a href="https://colab.research.google.com/github/fsternini/nlp-newspaper/blob/development/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# First, verification of Python version is completed
!which python # should return /usr/local/bin/python
!python --version

/usr/local/bin/python
Python 3.10.12


In [22]:
!git clone 'https://github.com/fsternini/nlp-newspaper.git'
%cd 'nlp-newspaper'
# Switch to the most updated branch
!git switch development
# Collect the most updated version of the repo
!git pull

Cloning into 'nlp-newspaper'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 4.63 KiB | 4.63 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/nlp-newspaper/nlp-newspaper/nlp-newspaper
Branch 'development' set up to track remote branch 'development' from 'origin'.
Switched to a new branch 'development'
Already up to date.


In [23]:
# Install all requirements for the code
!pip install -r requirements.txt -q
# Google Drive is mounted, to reach all relevant data
from google.colab import drive
import os
import pandas as pd
import colab_env

drive.mount('/content/drive')
dataset_path = os.environ['NLP_DATASET']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
eng_file_path = dataset_path+'/english_news.csv'
jpn_file_path = dataset_path+'/japanese_news.csv'
english_news = pd.read_csv(eng_file_path, sep='\t')
japanese_news = pd.read_csv(jpn_file_path, sep='\t')

<ipython-input-27-c97bfcc42ecb>:4: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  japanese_news = pd.read_csv(jpn_file_path, sep='\t')


In [29]:
english_news.head()

,source,date,title,author,text
0,Mainichi Shimbun,2021-12-06,"Editorial: Japan govt's 20,000-yen handout mis...",NaN,The Japanese government is trying to boost the...
1,Mainichi Shimbun,2021-12-07,Editorial: Time for Japan PM Kishida to delive...,NaN,"Prime Minister Fumio Kishida, who won a public..."
2,Mainichi Shimbun,2021-12-08,"Editorial: 80 years on from Pearl Harbor, acti...",NaN,It has been 80 years since the start of the Pa...
3,Mainichi Shimbun,2021-12-08,"Yoroku: 80 years later, many lessons still not...",NaN,"""The world has changed completely. The era was..."
4,Mainichi Shimbun,2021-12-08,Promoting respect for human rights behind US '...,NaN,The U.S. administration under President Joe Bi...


In [30]:
japanese_news.head()

,source,date,title,author,text
0,kobe-np.co.jp,2005-07-01,NaN,NaN,会見した北口寛人市長は「刑事訴訟で被告となっている職員にはそれぞれ主張があるが、組織全体とし...
1,kobe-np.co.jp,2005-07-01,NaN,NaN,明石・歩道橋事故をめぐる民事訴訟で、神戸地裁から計五億六千八百万円の賠償を命じられた兵庫県（...
2,kobe-np.co.jp,2007-04-07,NaN,NaN,会見後、遺族代理人の渡部吉泰弁護士は「裁判長が『問うべき者を問わないのは正義に反する』とはっ...
3,kobe-np.co.jp,2007-04-07,NaN,NaN,遺族会は、雑踏警備本部長を務めた元明石署長ら二人の起訴を求め、活動を続けている。
4,kobe-np.co.jp,2007-04-07,NaN,NaN,五人の遺族が閉廷後に会見。二女の優衣菜ちゃん＝当時（８つ）＝を亡くした三木清さん（３８）＝姫...


##English database Exploratory Data Analysis

In [32]:
english_news.count()

source    36889
date      36889
title     36889
author    12846
text      36766
dtype: int64

In [31]:
english_news.isna().sum()

source        0
date          0
title         0
author    24043
text        123
dtype: int64

In [34]:
english_news.drop('author',axis=1)

,source,date,title,text
0,Mainichi Shimbun,2021-12-06,"Editorial: Japan govt's 20,000-yen handout mis...",The Japanese government is trying to boost the...
1,Mainichi Shimbun,2021-12-07,Editorial: Time for Japan PM Kishida to delive...,"Prime Minister Fumio Kishida, who won a public..."
2,Mainichi Shimbun,2021-12-08,"Editorial: 80 years on from Pearl Harbor, acti...",It has been 80 years since the start of the Pa...
3,Mainichi Shimbun,2021-12-08,"Yoroku: 80 years later, many lessons still not...","""The world has changed completely. The era was..."
4,Mainichi Shimbun,2021-12-08,Promoting respect for human rights behind US '...,The U.S. administration under President Joe Bi...
...,...,...,...,...
36884,The Japan Times,2021-06-06,COVID-19 tracker: Tokyo reports 351 new cases ...,Tokyo confirmed 351 new cases of COVID-19 on S...
36885,The Japan Times,2021-06-07,Dollar weakens to around ¥109.40 in Tokyo,The dollar eased to around ¥109.40 in Tokyo tr...
36886,The Japan Times,2021-06-07,"Nikkei ends higher, but gains limited",The benchmark Nikkei average turned higher on ...
36887,The Japan Times,2021-06-07,COVID-19 tracker: Osaka reports 72 new cases b...,Osaka Prefecture reported 72 new COVID-19 case...


<Axes: >

Error in callback <function _draw_all_if_interactive at 0x7a3e788e9a20> (for post_execute):


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7a3e788e8ca0> (for post_execute):


KeyboardInterrupt: ignored